In [2]:
import plotly.graph_objs as go
import chart_studio.plotly as py

# Cufflinks wrapper on plotly
import cufflinks

# Data science imports
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

from plotly.offline import iplot

cufflinks.go_offline()

# Set global theme
cufflinks.set_config_file(world_readable=True, theme="pearl")

In [3]:

df = pd.read_csv("graduates.csv")
previous_rows, previous_columns = df.shape


df = df.dropna()
rows, columns = df.shape

print(f"{previous_rows - rows} row(s) dropped in dropna() step.")


1 row(s) dropped in dropna() step.


In [4]:
#Lägg till kolumnen ”ShareMen” som anger andelen män som jobbar på varje Major. (stödfunktion för apply)
def share_men(share):

    return 1 - share

#Lägg till antal män och kvinnor  som jobbar i respektive Major och se till att avrunda till integer. (stödfunktion för apply)
def amount_people(total, share, row):
    try:
        return int(total*share)
    except:
        print(f"NaN-values in following row: {list(row)}")
        return 0

In [5]:
#Lägg till kolumnen ”ShareMen” som anger andelen män som jobbar på varje Major.
df["ShareMen"] = df["ShareWomen"].apply(share_men)
df["Amount Men"] = df.apply(lambda row: amount_people(row['Total'], row['ShareMen'], row), axis=1)
df["Amount Women"] = df.apply(lambda row: amount_people(row['Total'], row['ShareWomen'], row), axis=1)


df.groupby("Major_category").sum()["Amount Men"].iplot(
    kind="bar",
    yTitle="Amount Men",
    linecolor="black",
    title="Amount Men",
)

In [6]:
#Plotta i histogram – först de 6 största, sedan sorterat på average av ShareWomen.

df.groupby("Major_category").mean()["ShareWomen"].nlargest(6).iplot(
    kind="bar",
    yTitle="Proportion Women",
    linecolor="black",
    title="Proportion of Women per Category",
)

In [7]:
#Plotta i histogram – först de 6 största, sedan sorterat på average av ShareWomen.
df.groupby("Major_category").mean()["ShareWomen"].sort_values(ascending = True).iplot(
    kind="bar",
    yTitle="Proportion Women",
    linecolor="black",
    title="Proportion of Women per Category",
)

In [8]:
df.groupby("Major_category").mean()["ShareWomen"].nlargest(4).iplot(
    kind="bar",
    yTitle="Number of Articles",
    linecolor="black",
    title="Articles by Publication",
)

In [9]:
tds = df[df["Major_category"] == "Engineering"]
#Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
tds.iplot(
    x="Unemployment_rate",
    y="Full_time_year_round",
    xTitle="Unemployment rate",
    yTitle="Amount of full time year round employees",
    text="Major",
    mode="markers",
    title="Job Prospects Engineering",
)

tds = df[df["Major_category"] == "Business"]
#Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
tds.iplot(
    x="Unemployment_rate",
    y="Full_time_year_round",
    xTitle="Unemployment rate",
    yTitle="Amount of full time year round employees",
    text="Major",
    mode="markers",
    title="Job Prospects Business",
)

In [10]:
df.iplot(
    x="Unemployment_rate",
    y="Full_time_year_round",
    categories="Major_category",
    xTitle="Unemployment rate",
    yTitle="Amount of full time year round employees",
    text="Major",
    mode="markers",
    title="Job Prospects",
)

/Users/evahegnar/opt/miniconda3/envs/visuell_env/lib/python3.10/site-packages/cufflinks/plotlytools.py:807: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.

/Users/evahegnar/opt/miniconda3/envs/visuell_env/lib/python3.10/site-packages/cufflinks/plotlytools.py:810: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.



In [11]:
df.iplot(
    x="Unemployment_rate",
    y="Full_time_year_round",
    categories="Major_category",
    size = 1.5*np.log(df["Total"]),
    xTitle="Unemployment rate",
    yTitle="Amount of full time year round employees",
    text="Major",
    mode="markers",
    title="Job Prospects",
)

In [12]:
import plotly.figure_factory as ff
corrs = df.corr()

figure = ff.create_annotated_heatmap(
    z=corrs.values,
    x=list(corrs.columns),
    y=list(corrs.index),
    colorscale="Earth",
    annotation_text=corrs.round(2).values,
    showscale=True,
    reversescale=True,
)

figure.layout.margin = dict(l=200, t=200)
figure.layout.height = 800
figure.layout.width = 1000

iplot(figure)

In [13]:
def shortener(text):
    return text[:4]
df["Short"] = df.apply(lambda row: shortener(row["Major_category"]),axis = 1)
figure = ff.create_scatterplotmatrix(
    df[["Short", "Median","Unemployment_rate"]],
    height=1000,
    width=1000,
    text=df["Major_category"],
    diag="histogram"
)
iplot(figure)
df.pop("Short")
print(df.head())

0      Engi
1      Engi
2      Engi
3      Engi
4      Engi
       ... 
168    Biol
169    Psyc
170    Psyc
171    Psyc
172    Educ
Name: Short, Length: 172, dtype: object

   Rank  Major_code                                      Major    Total  \
0     1        2419                      PETROLEUM ENGINEERING   2339.0   
1     2        2416             MINING AND MINERAL ENGINEERING    756.0   
2     3        2415                  METALLURGICAL ENGINEERING    856.0   
3     4        2417  NAVAL ARCHITECTURE AND MARINE ENGINEERING   1258.0   
4     5        2405                       CHEMICAL ENGINEERING  32260.0   

  Major_category  ShareWomen  Sample_size  Employed  Full_time  Part_time  \
0    Engineering    0.120564           36      1976       1849        270   
1    Engineering    0.101852            7       640        556        170   
2    Engineering    0.153037            3       648        558        133   
3    Engineering    0.107313           16       758       1069        150   
4    Engineering    0.341631          289     25694      23170       5180   

   Full_time_year_round  Unemployed  Unemployment_rate  Median  P25th   P75th  \
0    

In [14]:
print(df.head())
#Rank,Major_code,Major,Total,Major_category,ShareWomen,Sample_size,Employed,
# #Full_time,Part_time,Full_time_year_round,Unemployed,Unemployment_rate,Median,P25th,P75th,College_jobs,Non_college_jobs,Low_wage_jobs
data = [
    go.Scatter(
        x=grouped["P75th"],#grouped["word_count"],
        y=grouped["P25th"],#["reads"]
        text=grouped["Major"],#"title"]
        mode="markers",
        name=name,
        marker=dict(
            symbol=i,
            sizemin=2,
            sizeref=2,
            size=300*grouped["Unemployment_rate"],#["read_ratio"],
            colorscale="BlueRed",
            showscale=True,
            color=df["Sample_size"],#["read_time"]
            line=dict(color="black", width=1.2),
        ),
    )
    for i, (name, grouped) in enumerate(df.groupby("Major_category"))#groupby("publication")
]

figure = go.Figure(
    data=data,
    layout=go.Layout(
        legend=dict(x=1.2),
        width=1200,
        height = 1200,
        margin=dict(r=20),
        xaxis=dict(type="log", title="75th Wage Percentile"),
        yaxis=dict(title="25th Wage Percentile"),
        title="P25th vs P75th wage percentile Colored by Sample size, Sized by Unemployment Rate, and Shaped by Major Category",
    ),
)

iplot(figure)

   Rank  Major_code                                      Major    Total  \
0     1        2419                      PETROLEUM ENGINEERING   2339.0   
1     2        2416             MINING AND MINERAL ENGINEERING    756.0   
2     3        2415                  METALLURGICAL ENGINEERING    856.0   
3     4        2417  NAVAL ARCHITECTURE AND MARINE ENGINEERING   1258.0   
4     5        2405                       CHEMICAL ENGINEERING  32260.0   

  Major_category  ShareWomen  Sample_size  Employed  Full_time  Part_time  \
0    Engineering    0.120564           36      1976       1849        270   
1    Engineering    0.101852            7       640        556        170   
2    Engineering    0.153037            3       648        558        133   
3    Engineering    0.107313           16       758       1069        150   
4    Engineering    0.341631          289     25694      23170       5180   

   Full_time_year_round  Unemployed  Unemployment_rate  Median  P25th   P75th  \
0    